In [1]:
import pandas as pd
import math
import csv
import random
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import cross_val_score

In [2]:
# 当每支队伍没有elo等级分时，赋予其基础elo等级分
base_elo = 1600
team_elos = {} 
team_stats = {}
X = []
y = []
# 存放数据的目录
folder = 'data' 

In [3]:
# 根据每支队伍的Miscellaneous Opponent，Team统计数据csv文件进行初始化
def initialize_data(Mstat, Ostat, Tstat):
    new_Mstat = Mstat.drop(['Rk', 'Arena'], axis=1)
    new_Ostat = Ostat.drop(['Rk', 'G', 'MP'], axis=1)
    new_Tstat = Tstat.drop(['Rk', 'G', 'MP'], axis=1)

    team_stats1 = pd.merge(new_Mstat, new_Ostat, how='left', on='Team')
    team_stats1 = pd.merge(team_stats1, new_Tstat, how='left', on='Team')
    return team_stats1.set_index('Team', inplace=False, drop=True)

In [4]:
def get_elo(team):
    try:
        return team_elos[team]
    except:
        # 当最初没有elo时，给每个队伍最初赋base_elo
        team_elos[team] = base_elo
        return team_elos[team]

In [5]:
# 计算每个球队的elo值
def calc_elo(win_team, lose_team):
    winner_rank = get_elo(win_team)
    loser_rank = get_elo(lose_team)

    rank_diff = winner_rank - loser_rank
    exp = (rank_diff  * -1) / 400
    odds = 1 / (1 + math.pow(10, exp))
    # 根据rank级别修改K值
    if winner_rank < 2100:
        k = 32
    elif winner_rank >= 2100 and winner_rank < 2400:
        k = 24
    else:
        k = 16
    
    # 更新 rank 数值
    new_winner_rank = round(winner_rank + (k * (1 - odds)))      
    new_loser_rank = round(loser_rank + (k * (0 - odds)))
    return new_winner_rank, new_loser_rank

In [6]:
def  build_dataSet(all_data):
    print("Building data set..")
    X = []
    skip = 0
    for index, row in all_data.iterrows():

        Wteam = row['WTeam']
        Lteam = row['LTeam']

        #获取最初的elo或是每个队伍最初的elo值
        team1_elo = get_elo(Wteam)
        team2_elo = get_elo(Lteam)

        # 给主场比赛的队伍加上100的elo值
        if row['WLoc'] == 'H':
            team1_elo += 100
        else:
            team2_elo += 100

        # 把elo当为评价每个队伍的第一个特征值
        team1_features = [team1_elo]
        team2_features = [team2_elo]

        # 添加我们从basketball reference.com获得的每个队伍的统计信息
        for key, value in team_stats.loc[Wteam].iteritems():
            team1_features.append(value)
        for key, value in team_stats.loc[Lteam].iteritems():
            team2_features.append(value)

        # 将两支队伍的特征值随机的分配在每场比赛数据的左右两侧
        # 并将对应的0/1赋给y值
        if random.random() > 0.5:
            X.append(team1_features + team2_features)
            y.append(0)
        else:
            X.append(team2_features + team1_features)
            y.append(1)

        if skip == 0:
            print('X',X)
            skip = 1

        # 根据这场比赛的数据更新队伍的elo值
        new_winner_rank, new_loser_rank = calc_elo(Wteam, Lteam)
        team_elos[Wteam] = new_winner_rank
        team_elos[Lteam] = new_loser_rank

    return np.nan_to_num(X), y

In [7]:
def predict_winner(team_1, team_2, model):
    features = []

    # team 1，客场队伍
    features.append(get_elo(team_1))
    for key, value in team_stats.loc[team_1].iteritems():
        features.append(value)

    # team 2，主场队伍， 2020赛季没有观众故不加主场分
    features.append(get_elo(team_2))
    for key, value in team_stats.loc[team_2].iteritems():
        features.append(value)

    features = np.nan_to_num(features)
    return model.predict_proba([features])

In [8]:
if __name__ == '__main__':

    Mstat = pd.read_csv(folder + '/Miscellaneous.csv')
    Ostat = pd.read_csv(folder + '/Opponent.csv')
    Tstat = pd.read_csv(folder + '/Team.csv')

    team_stats = initialize_data(Mstat, Ostat, Tstat)

    result_data = pd.read_csv(folder + '/2020_result.csv')
    X, y = build_dataSet(result_data)

    # 训练网络模型
    print("Fitting on %d game samples.." % len(X))

    model = linear_model.LogisticRegression()
    model.fit(X, y)

    # 利用10折交叉验证计算训练正确率
    print("Doing cross-validation..")
    print(cross_val_score(model, X, y, cv = 10, scoring='accuracy', n_jobs=-1).mean())
    print('Predicting on new schedule..')
schedule21 = pd.read_csv(folder + '/2021_clean_schedule.csv')
result = []
for index, row in schedule21.iterrows():
    team1 = row['Vteam']
    team2 = row['Hteam']
    pred = predict_winner(team1, team2, model)
    prob = pred[0][0]
    if prob > 0.5:
        winner = team1
        loser = team2
        result.append([winner, loser, prob])
    else:
        winner = team2
        loser = team1
        result.append([winner, loser, 1 - prob])

with open('2021_Prediction_Result.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['win', 'lose', 'probability'])
    writer.writerows(result)
    print('done.')

Building data set..
X [[1600, 25.4, 30.0, 42.0, 33.0, 39.0, -1.29, 0.74, -0.55, 110.7, 111.9, -1.2, 103.7, 0.256, 0.40299999999999997, 0.568, 0.539, 13.9, 24.2, 0.18600000000000003, 0.532, 12.4, 77.8, 0.212, 528172.0, 16505.0, 42.7, 91.8, 0.465, 12.2, 33.9, 0.361, 30.5, 58.0, 0.526, 19.4, 24.7, 0.789, 10.1, 34.7, 44.8, 24.5, 8.5, 4.8, 14.5, 21.1, 117.1, 42.6, 91.6, 0.465, 13.6, 36.9, 0.37, 28.9, 54.8, 0.528, 17.1, 23.4, 0.7290000000000001, 11.1, 35.4, 46.5, 26.8, 7.5, 5.0, 16.4, 21.2, 115.8, 1700, 26.6, 53.0, 19.0, 50.0, 22.0, 6.24, -0.26, 5.97, 111.1, 105.0, 6.1, 100.9, 0.264, 0.42100000000000004, 0.574, 0.536, 13.1, 21.3, 0.21, 0.502, 14.6, 76.7, 0.20199999999999999, 633456.0, 19796.0, 37.8, 88.3, 0.428, 13.1, 38.9, 0.337, 24.7, 49.5, 0.499, 17.8, 23.2, 0.769, 10.9, 35.1, 46.1, 25.6, 7.2, 5.5, 16.8, 20.4, 106.5, 40.2, 87.9, 0.45799999999999996, 13.8, 37.0, 0.374, 26.4, 50.9, 0.519, 18.5, 23.2, 0.7959999999999999, 9.5, 35.9, 45.4, 25.2, 8.8, 5.0, 14.8, 21.7, 112.8]]
Fitting on 1060 ga